# Código para seleção e criação do modelo

In [1]:
import numpy as np
import pandas as pd

In [2]:
def stratified_sample(df, col='CATEGORY', n_per_class=2):
    return df.groupby(col, group_keys=False).apply(
        lambda x: x.sample(min(len(x), n_per_class)))

# Carregamento dos dados

In [3]:
data = pd.read_csv('../1.scraping/products.tsv', sep='\t')
data.set_index('ID', inplace=True)
data['SMARTPHONE'] = (data.CATEGORY == 'celular-e-smartphone').astype(int)

stratified_sample(data)

,CATEGORY,SOURCE,TITLE,SMARTPHONE
ID,,,,
325,aparelho-de-telefone,https://www.buscape.com.br/redirect_prod?id=37...,Telefone Intelbras Sem Fio Digital com Secreta...,0
189,aparelho-de-telefone,https://www.buscape.com.br/redirect_prod?id=37...,Telefone Digital de Mesa C / Fio VTC105W Branc...,0
834,capa-para-celular-e-smartphone,https://www.buscape.com.br/redirect_prod?id=46...,Capa para LG G2 Lite Gato 06 MP12908092,0
596,capa-para-celular-e-smartphone,https://www.buscape.com.br/redirect_prod?id=46...,Capa para Alcatel Pixi 4 3.5 Bailarina de Ball...,0
1109,celular-e-smartphone,https://www.buscape.com.br/smartphone-apple-ip...,Smartphone Apple iPhone SE 32GB,1
1319,celular-e-smartphone,https://www.buscape.com.br/smartphone-asus-zen...,Smartphone Asus ZenFone 4 Selfie ZD553KL 64GB ...,1
1790,gabinete,https://www.buscape.com.br/redirect_prod?id=11...,Gabinete ATX - AeroCool Cyclops ( c / janela )...,0
1476,gabinete,https://www.buscape.com.br/redirect_prod?id=11...,Gabinete Bertolini Evidence 4072 3 Portas 2 Ga...,0
1879,jogos-de-rpg,https://www.buscape.com.br/redirect_prod?id=63...,Cavalinho de Pelúcia Torcedor Flamengo,0


## Limpeza dos títulos

Devido a dificuldades em raspar de outras fontes, só pude coletar ofertas do buscapé.  
Um problema dessa fonte é que todos os produtos identificados como smartphones tinham a palavra smartphone ou celular no título.

In [4]:
print('Frequência das primeiras palavras de cada produto categorizado ' +
        f'como celular:')
phone_titles = data[data.CATEGORY == 'celular-e-smartphone'].TITLE
first_words = phone_titles.apply(lambda t: t.split()[0])

print(first_words.value_counts())

Frequência das primeiras palavras de cada produto categorizado como celular:
Smartphone    419
Celular        61
Name: TITLE, dtype: int64


Portanto realizei uma "limpeza" parcial dos títulos.  
Remover as palavras de todos os smartphones jogaria fora um atributo importante para a classificação,  
mas mantê-las seria quase incluir a variável resposta nas variáveis preditoras.

In [5]:
clean_titles = phone_titles.apply(lambda t: ' '.join(t.split()[1:]))
smartphones = data[data.CATEGORY == 'celular-e-smartphone']
trimmed_smartphones_idx = smartphones.sample(frac=0.4).index
data.loc[trimmed_smartphones_idx, 'TITLE'] = clean_titles

In [6]:
data[data.SMARTPHONE == 1].sample(10)

,CATEGORY,SOURCE,TITLE,SMARTPHONE
ID,,,,
1112,celular-e-smartphone,https://www.buscape.com.br/smartphone-motorola...,Smartphone Motorola Moto G 6 XT1925 32GB,1
1265,celular-e-smartphone,https://www.buscape.com.br/smartphone-xiaomi-r...,Xiaomi Redmi 6 64GB,1
1085,celular-e-smartphone,https://www.buscape.com.br/celular-bright-one-...,Bright One 0405,1
867,celular-e-smartphone,https://www.buscape.com.br/celular-samsung-key...,Samsung Keystone 2 GT-E1205,1
925,celular-e-smartphone,https://www.buscape.com.br/smartphone-lg-g3-d855,Smartphone LG G3 D855,1
1202,celular-e-smartphone,https://www.buscape.com.br/celular-dl-yc-330,Celular DL YC-330,1
870,celular-e-smartphone,https://www.buscape.com.br/celular-samsung-sgh...,Samsung SGH-F275,1
1074,celular-e-smartphone,https://www.buscape.com.br/smartphone-xiaomi-r...,Smartphone Xiaomi Redmi 5A 16GB MCE3B,1
1051,celular-e-smartphone,https://www.buscape.com.br/celular-dl-yc-120,DL YC-120,1


# Extração de features

Como na primeira parte do processo seletivo, tentei extrair features a partir de matches de expressões regulares.  
Todas as features definidas e expressões regulares correspondentes se encontram no arquivo `extract_features.py`.

In [7]:
from extract_features import get_df_attributes, attr_col_names

In [8]:
features = get_df_attributes(data)
stratified_sample(features)

,CATEGORY,SOURCE,TITLE,SMARTPHONE,re_smart,re_phone,re_celular,re_letra_num,re_capa,re_para,re_pelicula,re_plus,re_chip,re_MP
ID,,,,,,,,,,,,,,
222,aparelho-de-telefone,https://www.buscape.com.br/redirect_prod?id=37...,Telefone sem Fio KX - TGC220LBB Preto Dect 6.0...,0,0,1,0,1,0,0,0,0,0,0
316,aparelho-de-telefone,https://www.buscape.com.br/sem-fio-motorola-mo...,Sem Fio Motorola Moto 3000 MRD2,0,0,0,0,1,0,0,0,0,0,0
712,capa-para-celular-e-smartphone,https://www.buscape.com.br/redirect_prod?id=46...,Capa para Alcatel A3 5.0 Homem Aranha 01 MP127...,0,0,0,0,1,1,1,0,0,0,0
580,capa-para-celular-e-smartphone,https://www.buscape.com.br/redirect_prod?id=46...,Capa para iPhone 5C Partitura Musical 01 MP127...,0,0,1,0,1,1,1,0,0,0,0
1151,celular-e-smartphone,https://www.buscape.com.br/smartphone-asus-zen...,Smartphone Asus ZenFone 3 Zoom 64GB,1,1,1,0,0,0,0,0,0,0,0
1198,celular-e-smartphone,https://www.buscape.com.br/smartphone-asus-zen...,Smartphone Asus ZenFone Max Pro ZB602KL (M1) 32GB,1,1,1,0,1,0,0,0,0,0,0
1753,gabinete,https://www.buscape.com.br/redirect_prod?id=11...,Gabinete K - MEX GI - 9D89 1 Baias SLIM com Fonte,0,0,0,0,1,0,0,0,0,0,0
1576,gabinete,https://www.buscape.com.br/redirect_prod?id=11...,Gabinete Pcyes Spirit Full Window Duplo Preto ...,0,0,0,0,1,0,0,0,0,0,0
1838,jogos-de-rpg,https://www.buscape.com.br/redirect_prod?id=63...,Sauron - Hobbit 3 Lord Of The Rings Funko Pop ...,0,0,0,0,0,0,0,0,0,0,0


In [9]:
clf_features = features[attr_col_names + ['SMARTPHONE']]
stratified_sample(clf_features, col='SMARTPHONE', n_per_class=5)

,re_smart,re_phone,re_celular,re_letra_num,re_capa,re_para,re_pelicula,re_plus,re_chip,re_MP,SMARTPHONE
ID,,,,,,,,,,,
367,0,1,0,1,0,0,0,0,0,0,0
3763,1,0,0,1,0,0,0,0,0,0,0
564,0,0,0,1,1,1,0,0,0,0,0
822,0,0,0,1,1,1,0,0,0,0,0
3468,1,0,0,1,0,0,0,0,0,0,0
1251,1,1,0,1,0,0,0,0,0,0,1
1248,1,1,0,1,0,0,0,0,0,0,1
930,0,0,1,1,0,0,0,0,0,0,1
1107,1,1,0,1,0,0,0,0,0,0,1


# Comparação de classificadores

Foram comparados os desempenhos do Perceptron e Naive Bayes sobre os atributos extraídos acima, e o do Bag of Words diretamente com os títulos (já "limpos").

In [10]:
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_val_score

## Criação dos modelos

In [11]:
perceptron = Perceptron(max_iter=1e4)
naive_bayes = BernoulliNB(binarize=None)
bag_of_words = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('MNNB', MultinomialNB())
])

In [12]:
def eval_model(model, X, y):
    return cross_val_score(model, X, y, scoring='roc_auc', cv=10)


def print_results(name, cv_score):
    print(name)
    print(f'{np.mean(cv_score):.3f} +- {np.std(cv_score):.3f}')

## Avaliação dos resultados

In [13]:
perceptron_res = eval_model(perceptron, clf_features[attr_col_names].values, clf_features['SMARTPHONE'].values)

In [14]:
naive_bayes_res = eval_model(naive_bayes, clf_features[attr_col_names].values, clf_features['SMARTPHONE'].values)

In [15]:
bow_res = eval_model(bag_of_words, data.TITLE.values, data.SMARTPHONE.values)

In [16]:
print_results('Perceptron', perceptron_res)
print()
print_results('Naive Bayes', naive_bayes_res)
print()
print_results('Bag of Words', bow_res)

Perceptron
0.776 +- 0.188

Naive Bayes
0.801 +- 0.117

Bag of Words
0.993 +- 0.010


In [17]:
from scipy.stats import ttest_ind

test_result = ttest_ind(bow_res, naive_bayes_res)
print(f'p-value: {test_result.pvalue:.3f}')
if test_result.pvalue < 0.05:
    print(f'Classificadores com desempenhos distindos')
else:
    print(f'Não há evidências de que os classificadores tenham desempenhos diferentes')

p-value: 0.000
Classificadores com desempenhos distindos


## Escolha do modelo

Com base nos valores obtidos para a área sob a curva ROC e no teste de hipótese realizado acima, 
escolhi o modelo de Bag of Words para ser usado na classificação do dataset.

É válido notar que o formato e regularidade dos dados usados para treinamento não são representativos 
do conjunto dado como parte do teste, então mais testes rigorosos seriam necessários para determinar o melhor 
modelo. Porém, realizei testes superficiais e informais e o Bag of Words realmente parece ter o melhor desempenho dos 3.

# Determinação de threshold

O threshold foi escolhido novamente de maneira informal sobre o conjunto final.  
O valor fixado foi de `0.25`.

# Serialização do modelo

In [24]:
import pickle

In [19]:
X = data.TITLE.values
y = data.SMARTPHONE.values

In [25]:
model.fit(X, y)
with open('model.clf', 'wb+') as f:
    serialized_model = pickle.dump(model, f)